# LDA Demo

针对200篇名家散文抽取主题

## 载入相关扩展包

In [1]:
library(tidyverse)
library(jsonlite)
library(jiebaR)
library(lda)
library(LDAvis)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘jsonlite’

The following object is masked from ‘package:purrr’:

    flatten

Loading required package: jiebaRD


## 读入散文数据

In [2]:
prose <- read_json('../Data/prose.json', simplifyVector = T)

## 对散文进行中文分词

In [3]:
segmentor <- worker(stop_word = '../Data/stop-words-common.txt')
content_seg <- sapply(prose$content, segment, segmentor)

## 统计词频，通过词的长度和词频进行过滤

In [4]:
words_table <- table(unlist(content_seg))
words_table_sroted <- sort(words_table, decreasing = TRUE)
need_remove <- words_table_sroted < 5 | 
    str_length(names(words_table_sroted)) < 2 |
    str_detect(names(words_table_sroted), regex('[0-9a-zA-Z]+'))
words_table_sroted <- words_table_sroted[!need_remove]
vocab <- names(words_table_sroted)

## 构造用于LDA的文档

In [5]:
get_wrods <- function(x, vocab) {
    index <- match(x, vocab)
    index <- index[!is.na(index)]
    rbind(as.integer(index - 1), as.integer(rep(1, length(index))))
}

documents <- lapply(content_seg, get_wrods, vocab)

## 构建LDA模型

In [6]:
set.seed(112358)

k <- 10
max_iter <- 5000
alpha <- 0.1
eta <- 0.02

lda_model <- lda.collapsed.gibbs.sampler(
    documents = documents,
    K = k,
    vocab = vocab,
    num.iterations = max_iter,
    alpha = alpha,
    eta = eta,
    initial = NULL,
    burnin = 0,
    compute.log.likelihood = TRUE
)

## # 生成用于LDAVis可视化的JSON格式数据

In [7]:
vis_theta <- t(apply(lda_model$document_sums + alpha, 2, function(x) x / sum(x)))
vis_phi <- t(apply(t(lda_model$topics) + eta, 2, function(x) x / sum(x)))
vis_frequency <- as.integer(words_table_sroted)
vis_doc_length <- sapply(documents, function(x) sum(x[2, ]))
vis_json <- createJSON(
    phi = vis_phi, 
    theta = vis_theta,
    doc.length = vis_doc_length, 
    vocab = vocab,
    term.frequency = vis_frequency
)

## 生成LDAVis的可视化HTML页面

In [8]:
serVis(vis_json, out.dir = 'LDAVis', open.browser = F)
writeLines(iconv(readLines('LDAVis/lda.json'), from = 'GBK', to = 'UTF8'),
           file('LDAVis/lda.json', encoding='UTF-8'))

Warning message in dir.create(out.dir):
“'LDAVis' already exists”Loading required namespace: servr
Warning message in readLines("LDAVis/lda.json"):
“incomplete final line found on 'LDAVis/lda.json'”

## 查看不同主题的关键词

In [9]:
topic_words <- top.topic.words(lda_model$topics, num.words = 10)
colnames(topic_words) <- paste('topic', seq(1, 10), sep = '-')
rownames(topic_words) <- paste('word', seq(1, 10), sep = '-')

topic_words

,topic-1,topic-2,topic-3,topic-4,topic-5,topic-6,topic-7,topic-8,topic-9,topic-10
word-1,想到,母亲,教授,中国,朋友,世界,罪犯,女人,二月,地方
word-2,时间,父亲,学生,知识分子,三毛,生命,牛棚,男人,春天,中国
word-3,感到,儿子,德国,北大,读书,城市,地狱,陕西人,仿佛,想起
word-4,离开,孩子,老师,国民党,作家,孩子,劳改,闲人,地方,事情
word-5,一条,石头,中国,一点,西安,美丽,大院,奉承,荷花,榕树
word-6,东西,农民,离开,适之,副标题,柔和,人员,陕西,藤萝,故事
word-7,却是,两个,乔木,文章,作品,红柳,折磨,孔子,燕园,土地
word-8,心情,坐在,十年,胡适,写作,好像,胖子,有人,海棠,仙人掌
word-9,一位,一块,西克,印度,东西,明白,牢头,女子,花朵,广州
word-10,仿佛,说话,家里,思想,文章,旷野,禁子,英雄,叶子,江南


## 查看隶属于不同主题的文档

In [10]:
library(plyr)
topic_documents <- top.topic.documents(lda_model$document_sums, num.documents = 10)
topic_documents_detail <- aaply(topic_documents, c(1, 2), function(id) {
    paste(prose[id, ]$author, prose[id, ]$title, sep = ' - ')
})
colnames(topic_documents_detail) <- paste('topic', seq(1, 10), sep = '-')
rownames(topic_documents_detail) <- paste('prose', seq(1, 10), sep = '-')

topic_documents_detail

------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: ‘plyr’

The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following object is masked from ‘package:purrr’:

    compact



,topic-1,topic-2,topic-3,topic-4,topic-5,topic-6,topic-7,topic-8,topic-9,topic-10
prose-1,季羡林 - 八十述怀(2),贾平凹 - 我不是个好儿子,季羡林 - 迈耶一家,季羡林 - 为胡适说几句话(1),贾平凹 - 治病救人,贾平凹 - 对月,季羡林 - 在饥饿地狱中(2),贾平凹 - 陶俑,季羡林 - 幽径悲剧(1),秦牧 - 榕树的美髯
prose-2,季羡林 - 虎年抒怀(1),贾平凹 - 风雨,季羡林 - 迈耶一家,季羡林 - 一个老知识分子的心声(2),贾平凹 - 吃烟,毕淑敏 - 谎言三叶草,季羡林 - 在饥饿地狱中(2),贾平凹 - 陶俑,季羡林 - 二月兰(1),秦牧 - 花城
prose-3,季羡林 - 一条老狗(3),余光中 - 招魂的短笛--余光中,季羡林 - 我的老师们(3),秦牧 - 流氓经,贾平凹 - 《观云奇石》序,毕淑敏 - 鱼在波涛下微笑,季羡林 - 牛棚生活(5),毕淑敏 - 失去四肢的泳者,季羡林 - 二月兰(1),秦牧 - 花街十里一城春
prose-4,季羡林 - 新年抒怀(1),贾平凹 - 关于父子,余秋雨 - 诗歌文字,季羡林 - 一个老知识分子的心声(1),季羡林 - 虎年抒怀(3),毕淑敏 - 关于生命与命运的遐想,季羡林 - 牛棚生活(6),贾平凹 - 女人与陶瓶,季羡林 - 幽径悲剧(1),秦牧 - 去雾的家乡松石的画廊——赣东北三清山漫记
prose-5,季羡林 - 九十述怀(3),贾平凹 - 天马,季羡林 - 学习吐火罗文(1),季羡林 - 为胡适说几句话(2),贾平凹 - 好读书,贾平凹 - 两代人,季羡林 - 牛棚生活(9),贾平凹 - 石头沟里一位复退军人,季羡林 - 怀念西府海棠(1),余光中 - 春天，遂想起
prose-6,季羡林 - 新年抒怀(2),贾平凹 - 乡党王盛华,季羡林 - 我的老师们(2),季羡林 - 记张岱年先生,贾平凹 - 孙存蝶,毕淑敏 - 旷野与城市——毕淑敏,季羡林 - 牛棚生活(10),贾平凹 - 说奉承,季羡林 - 清塘荷韵(1),秦牧 - 神秘瑰丽的地下河
prose-7,季羡林 - 虎年抒怀(2),贾平凹 - 笑口常开,季羡林 - 我的老师们(1),季羡林 - 回忆陈寅恪先生(3),贾平凹 - 茶杯,毕淑敏 - 离太阳最近的树,季羡林 - 牛棚生活(2),贾平凹 - 关于女人,季羡林 - 幽径悲剧(2),秦牧 - 谈谈李莲英——一代杰出的奴才
prose-8,季羡林 - 牛棚杂忆——缘起(2),贾平凹 - 制造声音,季羡林 - 学习吐火罗文(2),季羡林 - 站在胡适之先生墓前(2),贾平凹 - 致李珖,毕淑敏 - 柔和——毕淑敏,季羡林 - 牛棚生活(3),贾平凹 - 听来的故事,季羡林 - 马缨花(2),秦牧 - 土地
prose-9,季羡林 - 一条老狗(1),季羡林 - 赋得永久的悔(1),季羡林 - 重返哥廷根(2),季羡林 - 我的心是一面镜子(9),贾平凹 - 朋友,毕淑敏 - 关于人生的沉思,季羡林 - 牛棚生活(1),贾平凹 - 进山东,季羡林 - 夹竹桃,秦牧 - 在遥远的海岸上
prose-10,张爱玲 - 精致女子之张爱玲,贾平凹 - 祭父,季羡林 - 我的心是一面镜子(6),季羡林 - 我的心是一面镜子(8),贾平凹 - 李相虎,毕淑敏 - 爱怕什么？,季羡林 - 牛棚杂忆——缘起(1),贾平凹 - 闲人,季羡林 - 新年抒怀(3),秦牧 - 仙人掌
